# CronJob management in kubenetes

CronJobs were first released by Bell labs in 1975. Used to invoke periodic jobs on a given day, week, or month at a specific hour & specific minute. CronJob kind has made its way into the kubenetes API and are packed full of capabilities not immediately noticeable by one's imagination. Here is a workflow I developed that functions well and also provides a quick way to debug by creating Jobs from CronJobs.

Lets begin defining a new CronJob. Design, write, push, and schedule an image to be ran in your k8 cluster using Docker Hub, Docker, and Python. We'll follow up design of the newly scheduled job, by checking logs and cleaning up our cluster. Removing unwanted Jobs, Pods, or other objects we've already checked.

`./sleeping-beetle.py`
```
#!/usr/bin/env python

import sys
import time

for idx in range(1, 5):
    print(f'{idx} beetle')
    time.sleep(.5)
    
sys.exit(0)
```

`./Dockerfile`
```
FROM debian:latest

RUN apt-get update && apt-get install python3-dev python3 -y

COPY ./sleeping-beetle.py ./sleeping-beetle.py

ENTRYPOINT ["python3", "sleeping-beetle.py"]
```

With `sleeping-beetle.py` and `Dockerfile` written, lets build a Docker Image and push it to Docker Hub.

```
$ docker build . -t sleeping-beetle
$ docker tag sleeping-beetle jbcurtin/sleeping-beetle
$ docker push jbcurtin/sleeping-beetle
The push refers to repository [docker.io/jbcurtin/sleeping-beetle]
94f1b0a94f4e: Layer already exists
519a13e31af7: Pushing [========================>                          ]  86.27MB/177MB
114ca5b7280f: Layer already exists
```

Sleeping Beetle is ready and available on Docker Hub, load it into a k8 cluster. To create a CronJob in a k8 cluster define `kind: CronJob` Object Type in kubernetes. Most of these Object Types are available across Cloud Provides such as Google Cloud Platform, Amazon Web Services, or Digital Ocean. Before we can create the new CronJob, we'll have to use a yaml file with adequate definition and apply it with `kubectl`.

`./sleeping-beetle.yaml`
```
apiVersion: batch/v1beta1
kind: CronJob
metadata:
  name: sleeping-beetle-cronjob
spec:
  schedule: '* */1 * * *'
  jobTemplate:
    spec:
      backoffLimit: 0
      template:
        spec:
          containers:
          - name: sleeping-beetle
            image: docker.io/jbcurtin/sleeping-beetle:latest
            imagePullPolicy: IfNotPresent
          restartPolicy: Never
```

Apply `sleeping-beetle.yaml` to the cluster
```
$ kubectl apply -f sleeping-beetle.yaml 
cronjob.batch/sleeping-beetle-cronjob created
```

Setup is complete, we can start creating objects with `kubectl`. Its about :44 in the afternoon and we don't want to wait for the next :00 to come about. Rather than fidgeting with `schedule:` attribute, we can create a `Job` from `cronjob.batch/sleeping-beetle-cronjob`

```
$ kubectl create job --from=cronjob/sleeping-beetle-cronjob sleeping-beetle-1
job.batch/sleeping-beetle-1 created
```

Check the logs

```
$ kubectl logs job/sleeping-beetle-1
1 beetle
2 beetle
3 beetle
4 beetle
```

Looks good. _Job_ complete. Clean up our cluster and logout.

```
$ kubectl delete jobs/sleeping-beetle-1
$ kubectl delete pod --field-selector=status.phase==Succeeded
$ exit
```